# add system start time to data by converting DoY and year

when we regularize and fill the gaps, then our data do not have system start time anymore.
We need this for linear interpolation stuff. Just lets add them in there.

In [2]:
import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import scipy
import scipy.signal
import os, os.path

from datetime import date
import datetime
import time

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb

import sys

In [3]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
import remote_sensing_core as rc
import remote_sensing_core as rcp

start_time = time.time()

In [5]:
data_dir = "/Users/hn/Documents/01_research_data/remote_sensing/"

param_dir = "/Users/hn/Documents/00_GitHub/Ag/remote_sensing/parameters/"

In [82]:
f_name = "01_Regular_filledGap_Grant_SF_2017_NDVI.csv"
a_df = pd.read_csv(data_dir + f_name, low_memory=False)


In [83]:
a_df.head(2)

,ID,Acres,county,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,Notes,RtCrpTy,Shap_Ar,Shp_Lng,TRS,image_year,SF_year,doy,NDVI,human_system_start_time
0,115184_WSDA_SF_2017,72.0,Grant,Orchard,apple,wsda,72.371199,2006/08/31,micro-sprinkler,2017/04/24,NaN,NaN,292875.850894,3308.620365,T13R24E4,2016,2017,213,0.685304,2016-07-31
1,115184_WSDA_SF_2017,72.0,Grant,Orchard,apple,wsda,72.371199,2006/08/31,micro-sprinkler,2017/04/24,NaN,NaN,292875.850894,3308.620365,T13R24E4,2016,2017,223,0.733553,2016-08-10


In [84]:
from calendar import timegm

utc_time = time.strptime(a_df.human_system_start_time[1], "%Y-%m-%d")
epoch_time = timegm(utc_time)
epoch_time = (epoch_time + 83000) * 1000
print (epoch_time)

print (time.strftime('%Y-%m-%d', time.localtime(epoch_time/1000)))

1470870200000
2016-08-10


In [85]:
epoch_vec = pd.to_datetime(a_df['human_system_start_time']).values.astype(np.int64) // 10 ** 6
epoch_vec = epoch_vec + 83000000
print (epoch_vec[1])
print (time.strftime('%Y-%m-%d', time.localtime(epoch_vec[1]/1000))) 

1470870200000
2016-08-10


In [86]:
utc_time = time.strptime(a_df.human_system_start_time[1], "%Y-%m-%d")
epoch_time = timegm(utc_time)
epoch_time

1470787200

In [87]:
def convert_human_system_start_time_to_systemStart_time(humantimeDF):
    epoch_vec = pd.to_datetime(humantimeDF['human_system_start_time']).values.astype(np.int64) // 10 ** 6

    # add 83000000 mili sec. since system_start_time is 1 day ahead of image_taken_time
    # that is recorded in human_system_start_time column.
    epoch_vec = epoch_vec + 83000000
    humantimeDF['system_start_time'] = epoch_vec

In [89]:
a_df.head(2)

,ID,Acres,county,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,...,RtCrpTy,Shap_Ar,Shp_Lng,TRS,image_year,SF_year,doy,NDVI,human_system_start_time,system_start_time
0,115184_WSDA_SF_2017,72.0,Grant,Orchard,apple,wsda,72.371199,2006/08/31,micro-sprinkler,2017/04/24,...,NaN,292875.850894,3308.620365,T13R24E4,2016,2017,213,0.685304,2016-07-31,1470006200000
1,115184_WSDA_SF_2017,72.0,Grant,Orchard,apple,wsda,72.371199,2006/08/31,micro-sprinkler,2017/04/24,...,NaN,292875.850894,3308.620365,T13R24E4,2016,2017,223,0.733553,2016-08-10,1470870200000


In [81]:
a_df